<a href="https://colab.research.google.com/github/noobmaster-ru/prak6_sem/blob/main/%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D1%83%D0%BC_6%D1%81%D0%B5%D0%BC%D0%B5%D1%81%D1%82%D1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Вычисления

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Play, VBox, HBox, FloatSlider
import re
import math
import plotly.graph_objects as go
from IPython.display import display, clear_output
import ipywidgets as widgets


# удаляем старые данные
!rm -rf data_files/data_M*.csv

# ==== Константы ====
# зафиксировать одну стационарную точку , один шаг eps = 0.5, один шаг step=0.01 и D >= 2*sqrt(M)
T0 = 0.0
TN = 200.0
TIME_STEP = 0.5
EPS = 0.5

# ==== Класс состояния ====
class State:
    def __init__(self, y1, y2, y3):
        self.y1 = y1
        self.y2 = y2
        self.y3 = y3

    def copy(self):
        return State(self.y1, self.y2, self.y3)

# ==== Вычисления начальных условий ====
def compute_stationary_solutions(M, D, sign_1, sign_2):
    y_star = State(0, 0, 0)
    discriminant = D**2 * M**2 - 4*M**3 + 3*M**2

    try:
        inner_sqrt = D*M + sign_2*np.sqrt(discriminant)

        if inner_sqrt < 0:
            print(f"Warning: Подкоренное выражение отрицательное: D={D}, M={M}, inner_sqrt={inner_sqrt}")
            inner_sqrt = 0  # Чтобы избежать ошибки, подставляем 0

        y_star.y2 = sign_1*np.sqrt(inner_sqrt)

        # Защита от деления на ноль
        if y_star.y2 == 0:
            print(f"Warning: Деление на ноль при расчете y1: D={D}, M={M}")
            y_star.y1 = 0
        else:
            y_star.y1 = -M / y_star.y2

        # Аналогично защищаем y3
        if y_star.y2 == 0:
            y_star.y3 = 0
        else:
            y_star.y3 = D/2 + (M/(y_star.y2**2)) * (1 - M)

    except Exception as e:
        print(f"Ошибка при расчете стационарных решений: {e}")
        y_star = State(0, 0, 0)  # Если что-то совсем пошло не так, обнуляем

    # y_star.y2 = -np.sqrt(D*M - np.sqrt(D**2 * M**2 - 4*M**3 + 3*M**2))
    # y_star.y1 = -M / y_star.y2
    # y_star.y3 = D/2 + M/(y_star.y2**2) * (1 - M)
    return y_star

def compute_beginning_values(vector_y_star):
    return State(
        vector_y_star.y1 + EPS,
        vector_y_star.y2 + EPS,
        vector_y_star.y3 + EPS
    )

# ==== Правая часть системы ====
def func(t, y, M, D):
    dy = State(0, 0, 0)

    dy.y1 = y.y1 - (D / 2.0) * y.y2 + y.y2 * (y.y3 + y.y1**2)
    dy.y2 = (D / 2.0) * y.y1 + y.y2 + y.y1 * (3 * y.y3 - y.y1**2)
    dy.y3 = -2.0 * y.y3 * (M + y.y1 * y.y2)
    return dy

# ==== Метод Рунге-Кутты 4-го порядка ====
def runge_kutta_4_system(M, D, sign_1, sign_2):
    vector_y_star = compute_stationary_solutions(M, D, sign_1, sign_2)
    vector_y_0 = compute_beginning_values(vector_y_star)
    t = T0
    y = vector_y_0
    results = []

    while (t <= TN + TIME_STEP):
        results.append((t, y.copy()))
        if (abs(y.y1) > 1e6 or abs(y.y2) > 1e6 or abs(y.y3) > 1e6):
          print(f"Warning: Решение ушло в бесконечность на t={t}")
          break

        k1 = func(t, y, M, D)
        k2 = func(t + TIME_STEP/2, State(
            y.y1 + TIME_STEP/2 * k1.y1,
            y.y2 + TIME_STEP/2 * k1.y2,
            y.y3 + TIME_STEP/2 * k1.y3
        ), M, D)
        k3 = func(t + TIME_STEP/2, State(
            y.y1 + TIME_STEP/2 * k2.y1,
            y.y2 + TIME_STEP/2 * k2.y2,
            y.y3 + TIME_STEP/2 * k2.y3
        ), M, D)
        k4 = func(t + TIME_STEP, State(
            y.y1 + TIME_STEP * k3.y1,
            y.y2 + TIME_STEP * k3.y2,
            y.y3 + TIME_STEP * k3.y3
        ), M, D)

        y.y1 += TIME_STEP/6 * (k1.y1 + 2*k2.y1 + 2*k3.y1 + k4.y1)
        y.y2 += TIME_STEP/6 * (k1.y2 + 2*k2.y2 + 2*k3.y2 + k4.y2)
        y.y3 += TIME_STEP/6 * (k1.y3 + 2*k2.y3 + 2*k3.y3 + k4.y3)

        t += TIME_STEP

    return results

# Построение графика

In [ ]:
def plot_trajectory(D,M,sign_1,sign_2):

  if (D >= np.sqrt(4*M-3)):
    results = runge_kutta_4_system(M,D,sign_1,sign_2)
    # Строим таблицу
    data = {
        'Time': [t for t, y in results],
        'y1': [y.y1 for t, y in results],
        'y2': [y.y2 for t, y in results],
        'y3': [y.y3 for t, y in results],
    }

    df = pd.DataFrame(data)

    fig = plt.figure(figsize=(16, 8))
    gs = gridspec.GridSpec(3, 2, width_ratios=[2, 1]) # 3 rows ,2 columns
    ax = fig.add_subplot(gs[:, 0], projection='3d')
    # plt.subplots_adjust(left=0.05, right=0.95, top=0.92, bottom=0.12, hspace=0.4)
    # Starting and ending points
    start = df.iloc[0]
    end = df.iloc[-1]

    # Drow the start and end points
    ax.scatter([start["Time"]],[start["y1"]], [start["y2"]], [start["y3"]], color="green", s=15, label="Begin point", zorder=5)
    ax.scatter([end["Time"]],[end["y1"]], [end["y2"]], [end["y3"]], color="red", s=15, label="End point", zorder=5)


    #  --- Projections start point ---
    # for y2-y3
    ax.plot([start["Time"],end["Time"]],[start["y1"], start["y1"]], [start["y2"], start["y2"]], [end["y3"], start["y3"]], color='green', linestyle='dashed')
    ax.scatter([end["Time"]],[start["y1"]], [start["y2"]], [start["y3"]], color="green", s=5, zorder=5)
    # for y1-y2
    ax.plot([start["Time"],start["Time"]],[start["y1"], start["y1"]], [start["y2"], end["y2"]], [start["y3"], start["y3"]], color='green', linestyle='dashed')
    ax.scatter([start["Time"]],[start["y1"]], [end["y2"]], [start["y3"]], color="green", s=5, zorder=5)
    # for y1-y3
    ax.plot([start["Time"],start["Time"]],[end["y1"], start["y1"]], [start["y2"], start["y2"]], [start["y3"], start["y3"]], color='green', linestyle='dashed')
    ax.scatter([start["Time"]],[end["y1"]], [start["y2"]], [start["y3"]], color="green", s=5, zorder=5)

    # --- Projections end point ---
    # for  y2-y3
    ax.plot([end["Time"],start["Time"]],[end["y1"], end["y1"]], [end["y2"], end["y2"]], [start["y3"], end["y3"]], color='red', linestyle='dashed')
    ax.scatter([start["Time"]],[end["y1"]], [end["y2"]], [end["y3"]], color="red", s=5, zorder=5)
    # for y1-y2
    ax.plot([end["Time"],end["Time"]],[end["y1"], end["y1"]], [end["y2"], start["y2"]], [end["y3"], end["y3"]], color='red', linestyle='dashed')
    ax.scatter([end["Time"]],[end["y1"]], [start["y2"]], [end["y3"]], color="red", s=5, zorder=5)
    # for y1-y3
    ax.plot([end["Time"],end["Time"]],[start["y1"], end["y1"]], [end["y2"], end["y2"]], [end["y3"], end["y3"]], color='red', linestyle='dashed')
    ax.scatter([end["Time"]],[start["y1"]], [end["y2"]], [end["y3"]], color="red", s=5, zorder=5)


    # Make trajectory for y1, y2, y3
    ax.plot(df["Time"],df["y1"], df["y2"], df["y3"], label="Trajectory", color="blue",linewidth=1.5)

    ax.set_xlabel("Y1", labelpad=10, fontsize=13,fontweight='bold')
    ax.set_ylabel("Y2", labelpad=10, fontsize=13,fontweight='bold')
    ax.set_zlabel("Y3", labelpad=10, fontsize=13,fontweight='bold')
    ax.set_title("График решения", fontsize=20)

    # Add legend and grid
    ax.legend(loc='best')
    ax.view_init(elev=40, azim=140)
    ax.grid(True)

    handles, labels = ax.get_legend_handles_labels()
    unique_labels = dict(zip(labels, handles))
    ax.legend(unique_labels.values(), unique_labels.keys())


    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(df["Time"], df["y1"], color='blue')
    ax2.set_xlabel("Time",fontsize=13,fontweight='bold')
    ax2.set_ylabel("y1".upper(), rotation=0, labelpad=13, fontsize=13,fontweight='bold')
    ax2.grid(True)

    ax2 = fig.add_subplot(gs[1, 1])
    ax2.plot(df["Time"], df["y2"], color='blue')
    ax2.set_xlabel("Time",fontsize=13,fontweight='bold')
    ax2.set_ylabel("y2".upper(), rotation=0, labelpad=13,fontsize=13,fontweight='bold')
    ax2.grid(True)

    ax2 = fig.add_subplot(gs[2, 1])
    ax2.plot(df["Time"], df["y3"], color='blue')
    ax2.set_xlabel("Time",fontsize=13,fontweight='bold')
    ax2.set_ylabel("y3".upper(), rotation=0, labelpad=13,fontsize=13,fontweight='bold')
    ax2.grid(True)

    #  Add text with constants to the left upper corner
    formula_str = (
        f'$y_2^* = {"" if sign_1 == 1 else "-"}\sqrt{{DM {"+ " if sign_2 == 1 else "- "} \sqrt{{D^2 M^2 - 4M^3 + 3M^2}}}}$'
    )
    textstr = '\n'.join((
        r'$D=%s$' % D,
        r'$M=%s$' % M,
        r'$T_0=%.1f$' % T0,
        r'$T_N=%.1f$' % TN,
        r'$\varepsilon=%.1f$' % EPS,
        r'$STEP=%.1f$' % TIME_STEP,
        formula_str
    ))
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)

    # Настройка отступов
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.1, hspace=0.4)
    plt.gcf().text(0.01, 0.8, textstr, fontsize=13, bbox=props)

    plt.show()
  else:
    print(f"Некоректные данные(D < sqrt(4*M-3)): {D} < sqrt(4*{M}-3)")

# Виджет - график по заданным данным

In [ ]:
# Слайдеры для
D_slider = widgets.FloatSlider( value=1.0, min=1.0, max=100.0, step=0.01, description='D')
M_slider = widgets.FloatSlider( value=1.01, min=1.01, max=100.0, step=0.01, description='M')
sign_1_slider = widgets.FloatSlider( value=1.00, min=-1.0, max=1.0, step=2.0, description='sign_1')
sign_2_slider = widgets.FloatSlider( value=1.00, min=-1.0, max=1.0, step=2.0, description='sign_2')

# Кнопка
button = widgets.Button(description='Вычислить', button_style='success')
# Виджет для вывода текста состояния
status = widgets.Label(value='')
# Отдельный output-блок для графика
output = widgets.Output()
# Функция, которая будет вызываться при нажатии на кнопку
def on_button_click(b):
    D = D_slider.value
    M = M_slider.value
    sign_1 = sign_1_slider.value
    sign_2 = sign_2_slider.value

    # Выводим сообщение "Идет расчет..."
    status.value = 'Идет расчет...'

    with output:
      clear_output(wait=True)
      plot_trajectory(D, M, sign_1, sign_2)
    # Обновляем статус
    status.value = 'Готово!'

# Связываем кнопку с функцией
button.on_click(on_button_click)
# Выводим всё на экран
ui = widgets.VBox([D_slider, M_slider, sign_1_slider, sign_2_slider, button, status, output])
display(ui)

# Значения D, M для перебора


In [ ]:
D_values = [1.00, 1.01, 1.02, 1.03, 1.04, 1.05, 1.1, 1.15, 1.2, 1.5, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0]
M_values = [1.01, 1.02, 1.03, 1.04, 1.05, 1.1, 1.2, 1.5, 2.0, 5.0]

# $ y_2^* = -\sqrt{A + \sqrt{B}}$

In [ ]:
for M in M_values:
  for D in D_values:
      # print(f'Рисуем график для D = {D}, M = {M}')
      plot_trajectory(D, M, -1, +1)
      print("\n")

# $y_2^* = \sqrt{A + \sqrt{B}}$

In [ ]:
for M in M_values:
  for D in D_values:
      print(f'Рисуем график для D = {D}, M = {M}')
      plot_trajectory(D, M, +1, +1)

# $y_2^* = \sqrt{A - \sqrt{B}}$

In [ ]:
for M in M_values:
  for D in D_values:
      print(f'Рисуем график для D = {D}, M = {M}')
      plot_trajectory(D, M, +1, -1)

# $y_2^* = -\sqrt{A - \sqrt{B}}$

In [ ]:
for M in M_values:
  for D in D_values:
      print(f'Рисуем график для D = {D}, M = {M}')
      plot_trajectory(D, M, -1, -1)